In [5]:
%%writefile mycppcode.cpp
#include <iostream>
#include <vector>
#include <iomanip>
#include <limits>
#include <chrono> // For measuring execution time
#include <random> // For generating random data
#include <omp.h>
#include <fstream> // For file output
#include <cstdlib>
#include <string>

using namespace std;
using namespace std::chrono;

void writeMPSFile(const string& filename, const vector<vector<double>>& A, const vector<double>& B, const vector<double>& C) {
	ofstream mpsFile(filename);
	if (!mpsFile.is_open()) {
		cerr << "Failed to open MPS file for writing." << endl;
		return;
	}

	int rows = A.size();
	int cols = A[0].size();

	// Write the header
	mpsFile << "NAME          LP_Problem" << endl;
	mpsFile << "OBJSENSE" << endl;
	mpsFile << "    MAX" << endl;
	mpsFile << "ROWS" << endl;
	mpsFile << " N  OBJ" << endl;
	for (int i = 0; i < rows; ++i) {
		mpsFile << " L  C" << i + 1 << endl;
	}

	// Write the columns
	mpsFile << "COLUMNS" << endl;
	for (int j = 0; j < cols; ++j) {
		for (int i = 0; i < rows; ++i) {
			if (A[i][j] != 0) {
				mpsFile << "    X" << j + 1 << "  C" << i + 1 << "  " << A[i][j] << endl;
			}
		}
		mpsFile << "    X" << j + 1 << "  OBJ  " << C[j] << endl;
	}

	// Write the RHS
	mpsFile << "RHS" << endl;
	for (int i = 0; i < rows; ++i) {
		mpsFile << "    RHS1  C" << i + 1 << "  " << B[i] << endl;
	}

	mpsFile << "ENDATA" << endl;
	mpsFile.close();
}

vector<vector<double>> generateLargeMatrix(int rows, int cols) {
	vector<vector<double>> matrix(rows, vector<double>(cols, 0.0));
	random_device rd;
	mt19937 gen(rd());
	uniform_real_distribution<> dis(0.0, 50.0);

	for (int i = 0; i < rows; ++i) {
		for (int j = 0; j < cols; ++j) {
			matrix[i][j] = dis(gen);
		}
	}

	return matrix;
}

vector<double> generateLargeVector(int size) {
	vector<double> vec(size, 0.0);
	random_device rd;
	mt19937 gen(rd());
	uniform_real_distribution<> dis(0.0, 50.0);

	for (int i = 0; i < size; ++i) {
		vec[i] = dis(gen);
	}

	return vec;
}

int selectPivotRow(int pivotColumn, int rows, vector<vector<double>>& tableau) {
	int pivotRow = -1, i;
	double minRatio = numeric_limits<double>::max();

#pragma omp parallel
	{
		int localPivotRow = -1;
		double localMinRatio = numeric_limits<double>::max();
#pragma omp for
		for (i = 0; i < rows; ++i) {
			if (tableau[i][pivotColumn] > 0) {
				double ratio = tableau[i].back() / tableau[i][pivotColumn];
				if (ratio < localMinRatio) {
					localMinRatio = ratio;
					localPivotRow = i;
				}
			}
		}
#pragma omp critical
		{
			if (localMinRatio < minRatio) {
				minRatio = localMinRatio;
				pivotRow = localPivotRow;
			}
		}
	}
	return pivotRow;
}

void pivot(int pivotRow, int pivotColumn, vector<vector<double>>& tableau, vector<int>& basicVars) {
	double pivotElement = tableau[pivotRow][pivotColumn];
	int i, j;

#pragma omp parallel for
	for (j = 0; j < tableau[0].size(); ++j) {
		tableau[pivotRow][j] /= pivotElement;
	}

#pragma omp parallel for
	for (i = 0; i < tableau.size(); ++i) {
		if (i != pivotRow) {
			double ratio = tableau[i][pivotColumn];
			for (int j = 0; j < tableau[0].size(); ++j) {
				tableau[i][j] -= ratio * tableau[pivotRow][j];
			}
		}
	}

	basicVars[pivotRow] = pivotColumn; // Update basic variable index
}

void printResults(int rows, int cols, vector<vector<double>>& tableau, vector<int>& basicVars) {
	cout << "Optimal Solution:" << endl;
	vector<double> variableValues(cols, 0.0);
	for (int i = 0; i < rows; ++i) {
		if (basicVars[i] < cols) {
			variableValues[basicVars[i]] = tableau[i].back();
		}
	}

	for (int i = 0; i < cols; ++i) {
		//used for verify small size data variable result
		if (variableValues[i] != 0) { //temporary put to no print if=0
			cout << "x" << i + 1 << " = " << fixed << setprecision(16) << variableValues[i] << endl;
		}
	}
	cout << "Maximum Z = " << fixed << setprecision(16) << tableau[rows].back() << endl;
}

int selectPivotColumn(int rows, int cols, vector<vector<double>>& tableau) {
	int pivotColumn = -1, j;
	double minValue = 0.0;

#pragma omp parallel
	{
		int localPivotColumn = -1;
		double localMinValue = 0.0;
#pragma omp for
		for (j = 0; j < cols; ++j) {
			if (tableau[rows][j] < localMinValue) {
				localMinValue = tableau[rows][j];
				localPivotColumn = j;
			}
		}

#pragma omp critical
		{
			if (localMinValue < minValue) {
				minValue = localMinValue;
				pivotColumn = localPivotColumn;
			}
		}
	}

	return pivotColumn;
}

long Simplex2(const vector<vector<double>>& A, const vector<double>& B, const vector<double>& C, int threadNum) {
	int rows = A.size();
	int cols = A[0].size();
	vector<vector<double>> tableau;
	vector<int> basicVars;
	tableau.resize(rows + 1, vector<double>(cols + rows + 1, 0.0));
	basicVars.resize(rows);

	omp_set_num_threads(threadNum);

	// Set up the tableau
#pragma omp parallel for
	for (int i = 0; i < rows; ++i) {
		for (int j = 0; j < cols; ++j) {
			tableau[i][j] = A[i][j];
		}
		tableau[i][cols + i] = 1; // Set the identity matrix for slack variables
		tableau[i].back() = B[i];
		basicVars[i] = cols + i; // Initialize basic variables (slack variables)
	}

#pragma omp parallel for
	for (int j = 0; j < cols; ++j) {
		tableau[rows][j] = -C[j];
	}
	tableau[rows].back() = 0; // Initialize the objective function's value

	auto start = high_resolution_clock::now();

	while (true) {
		int pivotColumn = selectPivotColumn(rows, cols, tableau);
		if (pivotColumn == -1) break; // Optimal solution found

		int pivotRow = selectPivotRow(pivotColumn, rows, tableau);
		if (pivotRow == -1) {
			cout << "The problem is unbounded." << endl;
			return 0.0;
		}

		pivot(pivotRow, pivotColumn, tableau, basicVars);
	}

	auto stop = high_resolution_clock::now();
	auto duration = duration_cast<microseconds>(stop - start);

	printResults(rows, cols, tableau, basicVars);
	cout << "Execution Time openMp: " << duration.count() << " microseconds" << endl;
	return duration.count();
}


//SERIAL BELOW
int selectPivotRowS(int pivotColumn, int rows, vector<vector<double>>& tableau) {
	int pivotRow = -1;
	double minRatio = numeric_limits<double>::max();


	int localPivotRow = -1;
	double localMinRatio = numeric_limits<double>::max();

	for (int i = 0; i < rows; ++i) {
		if (tableau[i][pivotColumn] > 0) {
			double ratio = tableau[i].back() / tableau[i][pivotColumn];
			if (ratio < localMinRatio) {
				localMinRatio = ratio;
				localPivotRow = i;
			}
		}
	}
	if (localMinRatio < minRatio) {
		minRatio = localMinRatio;
		pivotRow = localPivotRow;
	}


	return pivotRow;
}

void pivotS(int pivotRow, int pivotColumn, vector<vector<double>>& tableau, vector<int>& basicVars) {
	double pivotElement = tableau[pivotRow][pivotColumn];

	for (int j = 0; j < tableau[0].size(); ++j) {
		tableau[pivotRow][j] /= pivotElement;
	}

	for (int i = 0; i < tableau.size(); ++i) {
		if (i != pivotRow) {
			double ratio = tableau[i][pivotColumn];
			for (int j = 0; j < tableau[0].size(); ++j) {
				tableau[i][j] -= ratio * tableau[pivotRow][j];
			}
		}
	}

	basicVars[pivotRow] = pivotColumn; // Update basic variable index
}

void printResultsS(int rows, int cols, vector<vector<double>>& tableau, vector<int>& basicVars) {
	cout << "Optimal Solution:" << endl;
	vector<double> variableValues(cols, 0.0);
	for (int i = 0; i < rows; ++i) {
		if (basicVars[i] < cols) {
			variableValues[basicVars[i]] = tableau[i].back();
		}
	}

	for (int i = 0; i < cols; ++i) {
		//used for verify small size data variable result
		if (variableValues[i] != 0) { //temporary put to no print if=0
			cout << "x" << i + 1 << " = " << fixed << setprecision(16) << variableValues[i] << endl;
		}
	}
	cout << "Maximum Z = " << fixed << setprecision(16) << tableau[rows].back() << endl;
}

int selectPivotColumnS(int rows, int cols, vector<vector<double>>& tableau) {
	int pivotColumn = -1;
	double minValue = 0.0;

	int localPivotColumn = -1;
	double localMinValue = 0.0;

	for (int j = 0; j < cols; ++j) {
		if (tableau[rows][j] < localMinValue) {
			localMinValue = tableau[rows][j];
			localPivotColumn = j;
		}
	}

	if (localMinValue < minValue) {
		minValue = localMinValue;
		pivotColumn = localPivotColumn;
	}

	return pivotColumn;
}

long Simplex(const vector<vector<double>>& A, const vector<double>& B, const vector<double>& C) {
	int rows = A.size();
	int cols = A[0].size();
	vector<vector<double>> tableau;
	vector<int> basicVars;
	tableau.resize(rows + 1, vector<double>(cols + rows + 1, 0.0));
	basicVars.resize(rows);

	// Set up the tableau
	for (int i = 0; i < rows; ++i) {
		for (int j = 0; j < cols; ++j) {
			tableau[i][j] = A[i][j];
		}
		tableau[i][cols + i] = 1; // Set the identity matrix for slack variables
		tableau[i].back() = B[i];
		basicVars[i] = cols + i; // Initialize basic variables (slack variables)
	}

	for (int j = 0; j < cols; ++j) {
		tableau[rows][j] = -C[j];
	}
	tableau[rows].back() = 0; // Initialize the objective function's value

	auto start = high_resolution_clock::now(); // Start measuring time

	while (true) {
		int pivotColumn = selectPivotColumnS(rows, cols, tableau);
		if (pivotColumn == -1) break; // Optimal solution found

		int pivotRow = selectPivotRowS(pivotColumn, rows, tableau);
		if (pivotRow == -1) {
			cout << "The problem is unbounded." << endl;
			return 0.0;
		}

		pivotS(pivotRow, pivotColumn, tableau, basicVars);
	}

	auto stop = high_resolution_clock::now(); // Stop measuring time
	auto duration = duration_cast<microseconds>(stop - start);

	printResultsS(rows, cols, tableau, basicVars);
	cout << "Execution Time for Serial: " << duration.count() << " microseconds" << endl;
	return duration.count();
}


//ori
int main() {
	int numConstraints, numVariables;

	while (true) {
		cout << "Enter the number of constraints: ";
		if (cin >> numConstraints) {
			break;
		}
		else {
			cout << "Invalid input. Please enter an integer." << endl;
			cin.clear();
			cin.ignore(numeric_limits<streamsize>::max(), '\n');
		}
	}

	while (true) {
		cout << "Enter the number of variables: ";
		if (cin >> numVariables) {
			break;
		}
		else {
			cout << "Invalid input. Please enter an integer." << endl;
			cin.clear();
			cin.ignore(numeric_limits<streamsize>::max(), '\n'); // Discard invalid input
		}
	}

	// Generate random data for a large problem
	vector<vector<double>> A = generateLargeMatrix(numConstraints, numVariables);
	vector<double> B = generateLargeVector(numConstraints);
	vector<double> C = generateLargeVector(numVariables);

	// Write to MPS file for external solver
	writeMPSFile("problem.mps", A, B, C);

	ofstream outFile("performance_gains.txt");

	cout << "Serial (" << numConstraints << "*" << numVariables << ")->" << endl;
	long ori = Simplex(A, B, C);

	//cout << "2 threads(" << numConstraints << "*" << numVariables << ")->" << endl;
	//long time1 = Simplex2(A, B, C, 2);
	//double gain1 = static_cast<double>(ori) / time1;
	//outFile << "2 threads: " << gain1 << endl;

	//cout << "4 threads(" << numConstraints << "*" << numVariables << ")->" << endl;
	//long time2 = Simplex2(A, B, C, 4);
	//double gain2 = static_cast<double>(ori) / time2;
	//outFile << "4 threads: " << gain2 << endl;

	//cout << "8 threads(" << numConstraints << "*" << numVariables << ")->" << endl;
	//long time3 = Simplex2(A, B, C, 8);
	//double gain3 = static_cast<double>(ori) / time3;
	//outFile << "8 threads: " << gain3 << endl;

	cout << "16 threads(" << numConstraints << "*" << numVariables << ")->" << endl;
	long time4 = Simplex2(A, B, C, 16);
	double gain4 = static_cast<double>(ori) / time4;
	outFile << "16 threads: " << gain4 << endl;

	/*cout << "32 threads(" << numConstraints << "*" << numVariables << ")->" << endl;
	long time5 = Simplex2(A, B, C, 32);
	double gain5 = static_cast<double>(ori) / time5;
	outFile << "32 threads: " << gain5 << endl;*/

	outFile.close();

	//cout << "Performance Gain (2 threads vs serial for " << numConstraints << "x" << numVariables << "): " << fixed << setprecision(16) << static_cast<double>(ori) / time1 << endl;
	//cout << "Performance Gain (4 threads vs serial for " << numConstraints << "x" << numVariables << "): " << fixed << setprecision(16) << static_cast<double>(ori) / time2 << endl;
	//cout << "Performance Gain (8 threads vs serial for " << numConstraints << "x" << numVariables << "): " << fixed << setprecision(16) << static_cast<double>(ori) / time3 << endl;
	cout << "Performance Gain (16 threads vs serial for " << numConstraints << "x" << numVariables << "): " << fixed << setprecision(16) << static_cast<double>(ori) / time4 << endl;
	//cout << "Performance Gain (32 threads vs serial for " << numConstraints << "x" << numVariables << "): " << fixed << setprecision(16) << static_cast<double>(ori) / time5 << endl;

	// Write to MPS file for external solver
	//writeMPSFile("problem.mps", A, B, C);


	return 0;
}


Overwriting mycppcode.cpp


In [6]:
!g++ -fopenmp mycppcode.cpp -o mycppcode

In [ ]:
!./mycppcode

Enter the number of constraints: 20000
Enter the number of variables: 20000
Serial (20000*20000)->
Optimal Solution:
x1587 = 0.0001230543254115
x2042 = 0.0001447260349558
x2045 = 0.0000520616074080
x4704 = 0.0001444732766529
x5031 = 0.0000024746543143
x16577 = 0.0002260083310967
x19087 = 0.0000861950375231
Maximum Z = 0.0333878728151936
Execution Time for Serial: 383233955 microseconds
16 threads(20000*20000)->
Optimal Solution:
x1587 = 0.0001230543254115
x2042 = 0.0001447260349558
x2045 = 0.0000520616074080
x4704 = 0.0001444732766529
x5031 = 0.0000024746543143
x16577 = 0.0002260083310967
x19087 = 0.0000861950375231
Maximum Z = 0.0333878728151936
Execution Time openMp: 345558434 microseconds
Performance Gain (16 threads vs serial for 20000x20000): 1.1090279307146067


In [ ]:
!pip install highspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.1 MB/s eta 0:00:00


In [ ]:
# Import the HiGHS library
import highspy.highs as highs

mps_file_path = "/content/problem.mps"

try:
    # Create a HiGHS model
    model = highs.Highs()

    # Read the MPS file
    status = model.readModel(mps_file_path)

    if status != highs.HighsStatus.kOk:
        raise Exception("Failed to read the MPS file.")

    # Solve the model
    solve_status = model.run()

    if solve_status == highs.HighsStatus.kOk:
        print("Optimal Solution (non-zero variables):")
        solution = model.getSolution()

        # Loop through and print non-zero variables
        for idx, value in enumerate(solution.col_value):
            if value != 0:
                var_name = f"x{idx + 1}"
                print(f"{var_name} = {value}")

        # Print objective value
        print(f"Maximum Z = {model.getObjectiveValue()}")

    else:
        print("No optimal solution found or model did not solve successfully.")

except Exception as e:
    print(f"An error occurred: {e}")
